In [3]:
%pip install PDFPlumber 
%pip install openai
#%pip install pandas ipython
#%pip install os python-dotenv


import pdfplumber
import openai
import pandas as pd 
from typing import List, Dict
from IPython.display import display, Markdown
import os
from dotenv import load_dotenv



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Resume Summarization (PDF)
# Choose a resume (any sample PDF).

load_dotenv(override=True, dotenv_path="../.env.local")

# Initialize the client
from openai import OpenAI
my_api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=my_api_key)

# 1. Path to the local resume PDF
file_name = "./data/resume.pdf"

# 1. Open PDF Resume and Extract text from the PDF resume using PDFPlumber.
def extract_text_from_pdf(doc):
    with pdfplumber.open(doc) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
        return text
# Extract text from the resume
resume_text = extract_text_from_pdf(file_name)
print("Extracted Resume Text:")
print(resume_text[:500])  # Print first 500 characters of the resume text


Extracted Resume Text:
Nirmala Kumar
EXECUTIVE SUMMARY
Transformational Product Leader and MIT-trained CTO with 20+ years of experience leading enterprise digital
transformations and technical product delivery. Specialized in turning complex clinical and technical goals into
reliable, scalable, compliant systems. Proven track record in architectural strategy and platform modernization
for large-scale organizations (AT&T, Microsoft, Deloitte) and public sector entities. Currently Founder & CEO of
SpeakEQ, leveraging hi


In [15]:
# 2. Define a function to extract key information using OpenAI GPT.
from urllib import response
from openai import OpenAI

def analyze_resume_using_gpt(resume_text):
    # Added triple quotes for a multi-line string and fixed the f-string syntax
    prompt = f"""
    Extract the following information from the resume text:
    1. Candidate name
    2. Years of experience
    3. Skill set
    4. Education summary
    5. Generate a concise professional summary
    Resume Text: {resume_text}
"""
    client = OpenAI()

    # Using the modern Responses API for GPT-5
    response = client.chat.completions.create(
            model="gpt-5-nano", 
            messages=[
                {"role": "system", "content": "You are a resume optimization expert, specializing in authentic, strategic positioning of candidate experience. Your role is to analyze resumes against job descriptions to identify strengths, gaps, and keywords, ensuring content remains genuine and impactful for Applicant Tracking Systems (ATS) and hiring managers"},
                {"role": "user", "content": prompt}
                ]
        )

    print(response.choices[0].message.content)
    return response.choices[0].message.content

In [16]:
#Run the same prompt through:
#Local LLM

import ollama

def summarize_resume_with_local_llm(resume_text):
    prompt = f"""
    Extract the following information from the resume text:
    1. Candidate name
    2. Years of experience
    3. Skill set
    4. Education summary
    5. Generate a concise professional summary

    Resume Text:
    {resume_text}
    """
    # Call the Ollama model with JSON format
    response = local_llm_client = ollama.chat(
        model="llama3", 
        messages=[
            {"role": "system", "content": "You are a resume optimization expert, specializing in authentic, strategic positioning of candidate experience."
            "   Your role is to analyze resumes against job descriptions to identify strengths, gaps, and keywords, ensuring content remains genuine and impactful for Applicant Tracking Systems (ATS) and hiring managers"},
            {"role": "user", "content": prompt}
            ],
        format=None
    )

    print("Local LLM Response:")
    print(response)
    
    return response

In [12]:
#Bonus: Create a ranking function that picks the “best-fit” candidate for a specific job description based on extracted data.

def rank_candidate(resume_info, job_description):
    input_text = f"""
    Given the following resume information and job description, rate how well the candidate fits the job on a scale of 0 to 100.

    Resume Information:
    {resume_info}

    Job Description:
    {job_description}

    Provide only the numerical score.
    """
    response = client.responses.create(
        model="gpt-5-nano",
        reasoning={"effort": "minimal"}, 
        input=input_text,
        instructions="Rate candidate fit 0-100 based on resume and job description. Provide only the number."
        #max_tokens=10
    )
    score_text = response.output_text.strip()
    try:
        score = float(score_text)
    except ValueError:
        score = 0.0
    return score

In [17]:
# main function calling local llm 
import json

resume_file_path = './data/resume.pdf'
    
# Check if the file exists before proceeding
resume_text = extract_text_from_pdf(resume_file_path)
print("Extracted Resume Text:")
print(resume_text[:500])  # Print first 500 characters of the resume text

print("Parsing with Ollama...")
ollama_result = summarize_resume_with_local_llm(resume_text)

print("Parsing with GPT...")
gpt_result = analyze_resume_using_gpt(resume_text)


job_description = "The Role: We’re looking for a Head of AI Product who thrives in a fast-moving, high-ownership environment. " \
"You’ll help shape Nimbus’s core platform—expanding how users capture data, train models, and deploy automated agents at scale. " \
"This is a hands-on, high-impact role: part product strategist, part operator, and part builder. " \
"You’ll work across customer feedback, internal teams, and go-to-market to ensure Nimbus evolves into the most powerful self-learning, white-label AI platform on the market. " \
"What You’ll Do: Own the core product roadmap — focusing on data capture, training workflows, and agent automation. " \
"Partner with engineering and design to define, ship, and iterate on high-leverage features. Drive platform extensibility — e" \
"nabling partners and customers to embed, customize, and resell agents. Optimize onboarding and training loops to make data capture and model improvement frictionless." \
"Translate insights into action by setting KPIs, tracking adoption, and iterating based on real usage data.Bridge customer needs with technical realities, " \
"ensuring that what we build directly drives business outcomes.What We’re Looking For: 3–5+ years of product experience in SaaS or platform environments." \
"Strong understanding of AI, data, or automation-driven products (LLMs, conversational AI, or workflow tools). " \
"Experience with partner or platform ecosystems (APIs, integrations, white-label products).Ability to create compelling product narratives—both visually and strategically." \
"Technically conversant: comfortable working with APIs, webhooks, and developer-facing tools.Builder’s mindset: you move fast, embrace ambiguity, and balance vision with execution."

score = rank_candidate(resume_text, job_description)
print (f"Candidate Fit Score: {score}")

# Save the result to a text file
#with open("analysis_result.txt", "w", encoding="utf-8") as file:
#    file.write("ollama_result: \n")
#    file.write(ollama_resultchoices[0].message.content)
#    file.write("\n\n")
#    file.write("gpt_result: \n")
#    file.write(gpt_resultchoices[0].message.content)



Extracted Resume Text:
Nirmala Kumar
EXECUTIVE SUMMARY
Transformational Product Leader and MIT-trained CTO with 20+ years of experience leading enterprise digital
transformations and technical product delivery. Specialized in turning complex clinical and technical goals into
reliable, scalable, compliant systems. Proven track record in architectural strategy and platform modernization
for large-scale organizations (AT&T, Microsoft, Deloitte) and public sector entities. Currently Founder & CEO of
SpeakEQ, leveraging hi
Parsing with Ollama...
Local LLM Response:
model='llama3' created_at='2026-01-22T23:52:54.1842994Z' done=True done_reason='stop' total_duration=279686936700 load_duration=8082327500 prompt_eval_count=988 prompt_eval_duration=185419534100 eval_count=221 eval_duration=85895571200 message=Message(role='assistant', content="Based on the resume text, I extracted the following information:\n\n1. Candidate name: Nirmala Kumar\n2. Years of experience: 20+ years\n3. Skill set:\n\t